In [47]:
import tensorflow as tf 
from PIL import Image #python image library
import numpy as np
import os 
root = "training/"
filename="train.tfrecords"
def write_record(root,filename):
    writer=tf.python_io.TFRecordWriter(filename)
    for i in range(10):
        class_path = root+"Sample0{:02}/".format(i+1)
        for img_name in os.listdir(class_path):
            if img_name == '.DS_Store':
                continue
            img_path = class_path+img_name
            img = Image.open(img_path).convert("L") #轉成grayscale
            img_raw = img.tobytes()
            labels  = [0]*10
            labels[i] = 1
            #Example 裡面裝features
            #features裡面裝feature
            example=tf.train.Example(features=tf.train.Features(
              feature={
                "label":tf.train.Feature(int64_list=tf.train.Int64List(value=labels)),
                'img_raw':tf.train.Feature(bytes_list=tf.train.BytesList(
                                                                    value=[img_raw]))
            }))
            writer.write(example.SerializeToString())
    writer.close()        
write_record(root,filename)

In [54]:
def read_record(filename,batch_size,num_epochs):
    #建立檔名佇列
    filename_queue = tf.train.string_input_producer([filename],num_epochs=num_epochs)
    reader = tf.TFRecordReader()
    _,serialized_example = reader.read(filename_queue)
    #將「tf.train.Example」檔案解析為「tf.train.Features」
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label':tf.FixedLenFeature([10],tf.int64),
                                           'img_raw':tf.FixedLenFeature([],tf.string)
                                       })
    #將tobyte()改寫回來成tf.uint8
    img = tf.decode_raw(features['img_raw'],tf.uint8)
    img = tf.reshape(img,(128,128,1))
    img = tf.cast(img,tf.float32)/255-0.5
    label = tf.cast(features['label'],tf.float32)
    batch_xs,batch_ys = tf.train.shuffle_batch([img,label],batch_size=batch_size,
                        capacity=2000,min_after_dequeue=1000)
    #capacity是隊列的長度
    #min_after_dequeue是 隊列至少剩下min_after_dequeue個數據
    #min_after_dequeue越大，數據越亂
    return batch_xs,batch_ys  

In [74]:

#建立權重weight張量
def weight(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1),name='W')
#建立偏差(bias)張量
def bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape),name='b')
#定義conv2d函數，用於進行卷積運算
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
#建立max_pool_2*2函數
def max_pool_2_2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

with tf.name_scope('Input_Layer'):
    x = tf.placeholder("tf.float32",shape=[None,128,128,1],name="x")
with tf.name_scope('C1_Conv'):
    W1 = weight([5,5,1,16])
    b1 = bias([16])
    Conv1 = conv2d(x,W1) + b1
    C1_Conv = tf.nn.relu(Conv1)
with tf.name_scope('C1_Pool'):
    C1_Pool = max_pool_2_2(C1_Conv)
with tf.name_scope('C2_Conv'):
    W2 = weight([5,5,16,36])
    b2 = bias([36])
    Conv2 = conv2d(C1_Pool,W2)+b2
    C2_Conv = tf.nn.relu(Conv2)
with tf.name_scope('C2_Pool'):
    C2_Pool = max_pool_2_2(C2_Conv)
with tf.name_scope('D_Flat'):
    D_Flat = tf.reshape(C2_Pool,[-1,32*32*36])
    
with tf.name_scope('D_Hidden_Layer'):
    W3 = weight([32*32*36,128])
    b3 = bias([128])
    D_Hidden = tf.nn.relu(tf.matmul(D_Flat,W3)+b3)
    D_Hidden_Dropout = tf.nn.dropout(D_Hidden,keep_prob=0.8)
with tf.name_scope('Output_Layer'):
    W4 = weight([128,10])
    b4 = bias([10])
    y_predict = tf.nn.softmax(tf.matmul(D_Hidden_Dropout,W4)+b4)
with tf.name_scope("optimizer"):
    y_label = tf.placeholder("tf.float32",shape=[None,10],name="y_label")
    loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits
                                   (logits=y_predict,labels=y_label))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss_function)
    
with tf.name_scope("evaluate_model"):
    correct_prediction = tf.equal(tf.argmax(y_predict,1),tf.argmax(y_label,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"tf.float32"))

In [79]:
b_img,b_lab = read_record("train.tfrecords",10,20)
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    try:
        i=0
        while not coord.should_stop():
            i+=1
            img,lab = sess.run([b_img,b_lab])
            sess.run(optimizer,feed_dict={x:img,y_label:lab})
            if i%50 == 0:
                print("test")
                print("loss: ",sess.run(loss_function,feed_dict={x:img,y_label:lab}))
                print("pred: ",np.argmax(sess.run(y_predict,feed_dict={x:img}),axis=1))
                print("ans: " ,np.argmax(lab,axis=1))
    except tf.errors.OutOfRangeError:
        print("Done training")
        saver.save(sess,"./example.ckpt")
    finally:
        coord.request_stop()
        coord.join(threads)

Tensor("strided_slice_19:0", shape=(), dtype=uint8)
test
loss:  2.3608816
pred:  [5 5 5 5 5 5 5 5 5 5]
ans:  [6 4 7 7 7 7 5 4 2 7]
test
loss:  2.3521855
pred:  [5 5 5 7 5 5 5 5 5 5]
ans:  [1 6 5 9 2 2 4 0 3 0]
test
loss:  2.0562038
pred:  [7 7 7 5 7 3 3 5 7 3]
ans:  [7 4 7 2 7 4 8 0 2 3]
test
loss:  2.078605
pred:  [5 3 3 3 7 5 3 3 3 3]
ans:  [5 5 3 4 7 6 4 6 9 2]
test
loss:  2.2609138
pred:  [5 5 5 5 7 7 3 3 5 3]
ans:  [6 6 6 6 2 7 8 3 0 2]
test
loss:  1.8554935
pred:  [4 1 3 4 5 3 3 1 1 3]
ans:  [0 1 3 0 5 2 8 1 1 3]
test
loss:  1.8666184
pred:  [3 5 3 7 1 1 7 5 1 4]
ans:  [2 5 3 7 2 4 7 6 1 0]
test
loss:  1.9537203
pred:  [4 3 5 2 9 4 2 4 4 4]
ans:  [4 3 9 0 0 4 2 0 9 4]
test
loss:  1.6604881
pred:  [9 1 1 5 3 9 3 7 6 3]
ans:  [9 1 1 5 3 9 8 7 6 3]
test
loss:  1.4634016
pred:  [9 3 7 6 9 1 3 5 9 2]
ans:  [9 3 7 5 9 1 3 5 9 2]
test
loss:  1.5689378
pred:  [9 1 3 7 9 9 9 7 6 7]
ans:  [9 1 8 7 9 9 9 7 6 7]
test
loss:  1.9450058
pred:  [7 4 9 9 9 3 6 1 9 6]
ans:  [7 0 0 0 9 8 6 1 9 0]
t